# Homework 2
Chenjiaxin 1155246854

In [20]:
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance

In [21]:
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    vertexai=True,
    temperature=0
)

In [23]:
def encode_student_id(student_id: int) -> str:
    if student_id < 0:
        raise ValueError("student_id must be non-negative")
    M = 10**8
    a = 137
    b = 911
    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [24]:
encode_student_id(1155246854)

'68819909'

The main purpose of this step is to register an account. However, due to my mistake, I failed to save the API key and URL in time during the first run. ***Therefore, in order to complete the assignment, I changed my agent name.Now,my new agent name is JIAXINCHEN_68819909***

In [26]:
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "JIAXINCHEN_68819909", "description": "Agentic AI for FTEC5660"}'

{"success":true,"message":"Welcome to Moltbook! 🦞","agent":{"id":"dd72e1cf-6db5-4cd1-a1fb-b05e059e6fd5","name":"jiaxinchen_68819909","api_key":"moltbook_sk_2XXal3Ikg7WZOHXkuNTe-cSpTLXroQcv","claim_url":"https://www.moltbook.com/claim/moltbook_claim_vEVSXN78DgUdgZepdVVBqGs5PnKw-vsS","verification_code":"marine-PUSV","profile_url":"https://www.moltbook.com/u/jiaxinchen_68819909","created_at":"2026-02-24T06:16:24.960Z"},"setup":{"step_1":{"action":"SAVE YOUR API KEY","details":"Store it securely - you need it for all requests and it cannot be retrieved later!","critical":true},"step_2":{"action":"SET UP HEARTBEAT","details":"Add HEARTBEAT.md to your heartbeat routine so you check Moltbook periodically","url":"https://www.moltbook.com/heartbeat.md","why":"Without this, you'll never know when you're claimed or when someone replies to you!"},"step_3":{"action":"TELL YOUR HUMAN","details":"Send them the claim URL so they can verify you","message_template":"Hey! I just signed up for Moltbook, 

Here is the Moltbook tool function code, which encapsulates the website's interface into tools that can be called by the LLM Agent.

In [27]:
import os
import requests
from langchain_core.tools import tool
MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"
HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

Based on the teacher's code, the Agent has been enhanced with two additional capabilities: "subscribing to communities" and "viewing community information."

In [28]:
@tool
def subscribe_submolt(submolt_name: str) -> dict:
    """Subscribe to a submolt (community). Required for task: subscribe to /m/ftec5660."""
    if submolt_name.startswith('/m/'):
        submolt_name = submolt_name[3:]
    elif submolt_name.startswith('/'):
        submolt_name = submolt_name[1:]

    r = requests.post(
        f"{BASE_URL}/submolts/{submolt_name}/subscribe",
        headers=HEADERS,
        timeout=15
    )
    return r.json()
@tool
def get_submolt_info(submolt_name: str) -> dict:
    """Get information about a specific submolt."""
    if submolt_name.startswith('/m/'):
        submolt_name = submolt_name[3:]
    elif submolt_name.startswith('/'):
        submolt_name = submolt_name[1:]

    r = requests.get(
        f"{BASE_URL}/submolts/{submolt_name}",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

In [29]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent for FTEC5660 homework.
Your mission:
- Successfully complete the assigned tasks in order
- First, subscribe to /m/ftec5660
- Then, upvote and comment on the specified post
Rules:
1. Always check if previous actions succeeded before proceeding
2. Use meaningful comments about AI/ML/FinTech - no spam
3. If an action fails, try to understand why and report the error
4. Be professional and concise
5. Follow human instructions exactly
Available tools:
- get_feed: Browse recent posts
- search_moltbook: Search for content
- create_post: Create new posts
- comment_post: Comment on posts
- upvote_post: Upvote posts
- subscribe_submolt: Subscribe to communities (IMPORTANT for task 1)
- get_submolt_info: Get community details
Complete the tasks in this order:
1. Subscribe to /m/ftec5660
2. Upvote post 47ff50f3-8255-4dee-87f4-2c3637c7351c
3. Comment on that post with meaningful AI/FinTech insight
"""

This code can be used for automation and interaction systems on community platforms, such as automatically monitoring community activity and performing keyword-based searches, likes, or comments. It enables AI to handle repetitive tasks instead of manual operations, such as automatically subscribing to sub-communities, collecting activity data, or publishing content. Community administrators can instruct the agent to automatically execute operational tasks to improve efficiency. It can also be used for data collection and analysis before content moderation to help identify high-quality posts. Overall, it gives the community system a certain level of “autonomous execution capability.”

In [30]:
from typing import Any
def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")
def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"
def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", f"Starting Moltbook agent loop for {AGENT_NAME}")
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        api_key=GEMINI_VERTEX_API_KEY,
        vertexai=True,
    )
    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
        subscribe_submolt,
        get_submolt_info,
    ]
    agent = llm.bind_tools(tools)
    history = [
        {"role": "system", "content": SYSTEM_PROMPT}
    ]
    if instruction:
        history.append({"role": "human", "content": f"Human instruction: {instruction}"})
        log("HUMAN", instruction)
    else:
        history.append({"role": "human", "content": "Perform your Moltbook tasks."})
        log("AUTO", "No instruction - autonomous mode")
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()
        response = agent.invoke(history)
        history.append(response)
        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            if response.tool_calls:
                log("LLM.TOOL_CALLS", pretty(response.tool_calls))
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls - final answer produced in {elapsed}s")
            return response.content
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]
            log("TOOL", f"[{i}] Calling {tool_name}")
            log("TOOL.ARGS", pretty(args))
            tool_fn = None
            for t in tools:
                if t.name == tool_name:
                    tool_fn = t
                    break
            tool_start = time.time()
            try:
                if tool_fn:
                    result = tool_fn.invoke(args)
                    status = "success"
                else:
                    result = {"error": f"Tool {tool_name} not found"}
                    status = "error"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"
            tool_elapsed = round(time.time() - tool_start, 2)
            log("TOOL.RESULT", f"{tool_name} finished ({status}) in {tool_elapsed}s")

            if verbose:
                log("TOOL.OUTPUT", pretty(result))
            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=json.dumps(result, ensure_ascii=False)
                )
            )
        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."

The following is the completion status of the print task.

In [32]:
from datetime import datetime
import time
import json
from langchain_core.messages import ToolMessage
AGENT_NAME="jiaxinchen_68819909"
print(f"Starting agent: {AGENT_NAME}")
print("\nTASK 1: Subscribe to /m/ftec5660")
print("-"*30)
result1 = moltbook_agent_loop("Subscribe to submolt ftec5660", max_turns=4)
print(f"\nTask 1 result:\n{result1}")
time.sleep(2)
print("\nTASK 2: Upvote and comment on the specified post")
print("-"*30)
target_post = "47ff50f3-8255-4dee-87f4-2c3637c7351c"
instruction = f"""
Please complete these actions in order:
1. Upvote the post with ID: {target_post}
2. Add a meaningful comment to the same post about AI in FinTech or Agentic AI
Make sure to check if each action succeeds before moving to the next.
"""
result2 = moltbook_agent_loop(instruction, max_turns=6)
print(f"\nTask 2 result:\n{result2}")
print("\nAll tasks completed!")

Starting agent: jiaxinchen_68819909

TASK 1: Subscribe to /m/ftec5660
------------------------------
[06:27:53] [INIT] Starting Moltbook agent loop for jiaxinchen_68819909


/tmp/ipython-input-1230684194.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[06:27:53] [HUMAN] Subscribe to submolt ftec5660
[06:27:53] [TURN] Turn 1/4 started
[06:27:54] [LLM] Model responded
[06:27:54] [LLM.CONTENT] <empty>
[06:27:54] [LLM.TOOL_CALLS] [
  {
    "name": "subscribe_submolt",
    "args": {
      "submolt_name": "ftec5660"
    },
    "id": "63952f21-4b87-4459-aa91-8007af9bd163",
    "type": "tool_call"
  }
]
[06:27:54] [TOOL] [1] Calling subscribe_submolt
[06:27:54] [TOOL.ARGS] {
  "submolt_name": "ftec5660"
}
[06:27:55] [TOOL.RESULT] subscribe_submolt finished (success) in 0.54s
[06:27:55] [TOOL.OUTPUT] {
  "success": true,
  "message": "Subscribed to m/ftec5660! 🦞",
  "action": "subscribed"
}
[06:27:55] [TURN] Turn 1 completed in 1.66s
[06:27:55] [TURN] Turn 2/4 started
[06:27:55] [LLM] Model responded
[06:27:55] [LLM.CONTENT] <empty>
[06:27:55] [LLM.TOOL_CALLS] [
  {
    "name": "upvote_post",
    "args": {
      "post_id": "47ff50f3-8255-4dee-87f4-2c3637c7351c"
    },
    "id": "5f13cc40-44fb-48fe-a2ab-5429e36c55ce",
    "type": "tool_call"
